<a href="https://colab.research.google.com/github/ptaceda/mrtva_doba_VU/blob/main/zpracovani_vysledku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pydicom # type: ignore
import matplotlib.pyplot as plt # type: ignore
import pandas as pd # type: ignore
import os
from glob import glob



In [3]:
## OPTIMA prvotni DATA
optima_folder = 'Optima'

optima_mds = glob(os.path.join(optima_folder, 'MD*'))

for md in optima_mds:
    text_file_path = os.path.join(md, 'vysledky.txt')
    with open(text_file_path, 'w+') as file:
        file.write('Typ_Datum_Kamera    Hlava_okno   Count_rate\n')

    dead_time_path = glob(os.path.join(md, 'MD_*.dcm'))[0]
    pozadi_path = glob(os.path.join(md, 'Pozadi_*.dcm'))[0]
    data_paths = [dead_time_path, pozadi_path]
    
    dt_data = pydicom.dcmread(dead_time_path)
    pozadi_data = pydicom.dcmread(pozadi_path)
    datas = [dt_data, pozadi_data]

    for dicom_data, path in zip(datas, data_paths):
        # Get the base filename without extension
        base_filename = os.path.basename(path)
        filename_without_extension = os.path.splitext(base_filename)[0]

        # Access the custom group containing Dataset Names and print them
        if (0x0011, 0x1012) in dicom_data:
            dataset_names = dicom_data[0x0011, 0x1012].value
            dataset_acq_time = dicom_data[0x0018, 0x1242].value*0.001

            # Extract the pixel array from the DICOM data
            if 'PixelData' in dicom_data:
                pixel_array = dicom_data.pixel_array

                # Check if the pixel array has the expected number of slices
                num_slices = pixel_array.shape[0]
                if num_slices != len(dataset_names):
                    print("Warning: Number of slices does not match the number of dataset names.")

                # Create a figure and a 2x3 grid of subplots
                fig, axes = plt.subplots(2, 3, figsize=(15, 10), dpi = 200)
                
                # Flatten the axes array for easy iteration
                axes = axes.flatten()

                # Separate the indices of "Head1" and "Head2" datasets
                head1_indices = [i for i, name in enumerate(dataset_names) if "Head1" in name]
                head2_indices = [i for i, name in enumerate(dataset_names) if "Head2" in name]

                head1_dict = {}
                head2_dict = {}

                # Loop through each "Head1" slice and plot it in the first row
                for idx, head1_idx in enumerate(head1_indices):
                    ax = axes[idx]
                    ax.imshow(pixel_array[head1_idx], cmap='gray')
                    ax.set_title(dataset_names[head1_idx])
                    ax.axis('off')  # Hide the axes ticks

                    # Calculate and print the sum of pixel values
                    pixel_sum = pixel_array[head1_idx].sum()
                    count_rate = pixel_sum / dataset_acq_time
                    with open(text_file_path, 'a+') as file:
                        file.write(f"{filename_without_extension}   {dataset_names[head1_idx]}   {round(count_rate,2)}\n")

                # Loop through each "Head2" slice and plot it in the second row
                for idx, head2_idx in enumerate(head2_indices):
                    ax = axes[idx + 3]
                    ax.imshow(pixel_array[head2_idx], cmap='gray')
                    ax.set_title(dataset_names[head2_idx])
                    ax.axis('off')  # Hide the axes ticks

                    # Calculate and print the sum of pixel values
                    pixel_sum = pixel_array[head2_idx].sum()
                    count_rate = pixel_sum / dataset_acq_time
                    with open(text_file_path, 'a+') as file:
                        file.write(f"{filename_without_extension}   {dataset_names[head2_idx]}  {round(count_rate,2)}\n")

                # Adjust layout to avoid overlap
                fig.savefig(os.path.join(md, f'{filename_without_extension}.jpg'), bbox_inches='tight')
                plt.close()
            else:
                print("No image data found in the DICOM file.")
        else:
            print("The DICOM file does not contain the expected custom group with dataset names.")
    



    

['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']
['Head1_EM', 'Head2_EM', 'Head1_SC1', 'Head2_SC1', 'Head1_SC2', 'Head2_SC2']

In [4]:
## DISCOVERY prvotni DATA
discovery_folder = 'Discovery'
a = 3

discovery_mds = glob(os.path.join(discovery_folder, 'MD*'))

for md in discovery_mds:
    text_file_path = os.path.join(md, 'vysledky.txt')
    with open(text_file_path, 'w+') as file:
        file.write('Vysledky mereni mrtve doby:\n')
        file.write('Typ_Datum_kamera    Hlava_okno  Pocet_akum_impulsu  Acq_time  Count_rate\n')

    dead_time_path = glob(os.path.join(md, 'MD_*.dcm'))[0]
    pozadi_path = glob(os.path.join(md, 'Pozadi_*.dcm'))[0]
    data_paths = [dead_time_path, pozadi_path]
    
    dt_data = pydicom.dcmread(dead_time_path)
    pozadi_data = pydicom.dcmread(pozadi_path)
    datas = [dt_data, pozadi_data]

    for dicom_data, path in zip(datas, data_paths):
        # Get the base filename without extension
        base_filename = os.path.basename(path)
        filename_without_extension = os.path.splitext(base_filename)[0]

        # Access the custom group containing Dataset Names and print them
        if (0x0011, 0x1012) in dicom_data:
            dataset_names = dicom_data[0x0011, 0x1012].value
            dataset_acq_time = dicom_data[0x0018, 0x1242].value*0.001

            # Extract the pixel array from the DICOM data
            if 'PixelData' in dicom_data:
                pixel_array = dicom_data.pixel_array

                # Check if the pixel array has the expected number of slices
                num_slices = pixel_array.shape[0]
                if num_slices != len(dataset_names):
                    print("Warning: Number of slices does not match the number of dataset names.")

                # Create a figure and a 2x3 grid of subplots
                fig, axes = plt.subplots(2, 3, figsize=(15, 10), dpi = 200)
                
                # Flatten the axes array for easy iteration
                axes = axes.flatten()

                # Separate the indices of "Head1" and "Head2" datasets
                head1_indices = [i for i, name in enumerate(dataset_names) if "Head1" in name]
                head2_indices = [i for i, name in enumerate(dataset_names) if "Head2" in name]

                # Loop through each "Head1" slice and plot it in the first row
                for idx, head1_idx in enumerate(head1_indices):
                    ax = axes[idx]
                    ax.imshow(pixel_array[head1_idx], cmap='gray')
                    ax.set_title(dataset_names[head1_idx])
                    ax.axis('off')  # Hide the axes ticks

                    # Calculate and print the sum of pixel values
                    pixel_sum = pixel_array[head1_idx].sum()
                    count_rate = pixel_sum / dataset_acq_time
                    with open(text_file_path, 'a+') as file:
                        file.write(f"{filename_without_extension}   {dataset_names[head1_idx]}  {pixel_sum} {dataset_acq_time} {round(count_rate,2)}\n")

                # Loop through each "Head2" slice and plot it in the second row
                for idx, head2_idx in enumerate(head2_indices):
                    ax = axes[idx + 3]
                    ax.imshow(pixel_array[head2_idx], cmap='gray')
                    ax.set_title(dataset_names[head2_idx])
                    ax.axis('off')  # Hide the axes ticks

                    # Calculate and print the sum of pixel values
                    pixel_sum = pixel_array[head2_idx].sum()
                    count_rate = pixel_sum / dataset_acq_time
                    with open(text_file_path, 'a+') as file:
                        file.write(f"{filename_without_extension}   {dataset_names[head2_idx]} {pixel_sum} {dataset_acq_time} {round(count_rate,2)}\n")
                        file.close()

                # Adjust layout to avoid overlap
                fig.savefig(os.path.join(md, f'{filename_without_extension}.jpg'), bbox_inches='tight')
                plt.close()
            else:
                print("No image data found in the DICOM file.")
        else:
            print("The DICOM file does not contain the expected custom group with dataset names.")
    

In [3]:
# ZKOUŠKA JESTLI VŠE FUNGUJE A FUNGUJE

from dicom_file_separator import separate_dicom_file
from tew_correction import tew_correction
import numpy as np


path = "Discovery/MD_08072024_povedene/MD_08072024_Discovery.dcm"
images = separate_dicom_file(path)
tew_1 = tew_correction(images["Head1_EM"], images["Head1_SC1"], images["Head1_SC2"])

bez_korekce = np.sum(images["Head1_EM"])/images["Acq_time"]
s_korekci = np.sum(tew_1)/images["Acq_time"]

print(bez_korekce, s_korekci)

26751.11111111111 15559.776905017536


In [18]:
# DISCOVERY DATA mrtve doby

from dicom_file_separator import separate_dicom_file
from tew_correction import tew_correction
import numpy as np



discovery_folder = 'Discovery'
discovery_mds = glob(os.path.join(discovery_folder, 'MD*'))

for md in discovery_mds:
    ### nejprve rozhazeni jednotlivych dat
    dead_time_path = glob(os.path.join(md, 'MD_*.dcm'))[0]
    pozadi_path = glob(os.path.join(md, 'Pozadi_*.dcm'))[0]

    dt_data = separate_dicom_file(dead_time_path)
    bg_data = separate_dicom_file(pozadi_path)

    dt_acq_time = dt_data["Acq_time"]
    bg_acq_time = bg_data["Acq_time"]

    dt_1_no_corr = dt_data["Head1_EM"]
    bg_1_no_corr = bg_data["Head1_EM"]

    dt_1_tew_corr = tew_correction(dt_data["Head1_EM"], dt_data["Head1_SC1"], dt_data["Head1_SC2"])
    bg_1_tew_corr = tew_correction(bg_data["Head1_EM"], bg_data["Head1_SC1"], bg_data["Head1_SC2"])

    dt_2_no_corr = dt_data["Head2_EM"]
    bg_2_no_corr = bg_data["Head2_EM"]
    
    dt_2_tew_corr = tew_correction(dt_data["Head2_EM"], dt_data["Head2_SC1"], dt_data["Head2_SC2"])
    bg_2_tew_corr = tew_correction(bg_data["Head2_EM"], bg_data["Head2_SC1"], bg_data["Head2_SC2"])

    ### ----
    ### vypocty count rates

    cr_1_no_corr = np.sum(dt_1_no_corr)/dt_acq_time - np.sum(bg_1_no_corr)/bg_acq_time
    cr_2_no_corr = np.sum(dt_2_no_corr)/dt_acq_time - np.sum(bg_2_no_corr)/bg_acq_time

    cr_1_tew_corr = np.sum(dt_1_tew_corr)/dt_acq_time - np.sum(bg_1_tew_corr)/bg_acq_time
    cr_2_tew_corr = np.sum(dt_2_tew_corr)/dt_acq_time - np.sum(bg_2_tew_corr)/bg_acq_time

    print(np.sum(dt_1_no_corr)/dt_acq_time, np.sum(bg_1_no_corr)/bg_acq_time)
    print(np.sum(dt_1_tew_corr)/dt_acq_time, np.sum(bg_1_tew_corr)/bg_acq_time)






26751.11111111111 23.801666666666666
15559.776905017536 20.146251051179096
26606.736166800318 24.028333333333332
16049.818140091667 20.20515628516268
26107.838827838827 24.305
15556.302832772293 20.462846948323445
25585.600439681228 23.946666666666665
15200.334877492394 20.184489156569427
24720.75325680382 24.268333333333334
14668.898230998851 20.460646282675384
22495.455043259226 24.121666666666666
13367.689817542558 20.25380017529522
21865.120632279533 24.30666666666667
12986.278269797884 20.43844763055348
19896.60172179429 23.89
11787.961060607662 19.9601061246728
17587.416352588265 24.065
10543.843657277677 20.163234137559368
15563.192312825304 24.205
9299.972468589154 20.52445368297623
